In [1]:
# coding: utf-8

from  __future__ import absolute_import
from __future__ import print_function
from ImageDataGeneratorCustom import ImageDataGeneratorCustom
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
from keras import optimizers
import shutil
import random
from PIL import Image
import numpy as np
import logging
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
import tensorflow_hub as hub
import shutil
import random
from PIL import Image
import pickle
import shutil
from tensorflow import debugging
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet
from keras.callbacks import ModelCheckpoint

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


print(tf.__version__)

# tf.config.run_functions_eagerly(True)
# tf.data.experimental.enable_debug_mode()

model_path = "./deep_ranking"

# batch_size = 96
# batch_size = 8
batch_size = 24
# batch_size = 3
# batch_size = 1

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


2.8.0


# Model

a lot of the code comes from https://keras.io/examples/vision/siamese_network/
and https://github.com/akarshzingade/image-similarity-deep-ranking/blob/master/deepRanking.py

In [2]:


embedding = tf.keras.Sequential([

    # GlobalAveragePooling2D(),

    tf.keras.layers.Dense(1024, activation='linear'),


    # tf.keras.layers.Dense(4096, activation='relu'),
    # tf.keras.layers.Dropout(0.6),
    # tf.keras.layers.Dense(4096, activation='relu'),
    # tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
])

class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)


anchor_input = layers.Input(name="anchor", shape=(1280))
positive_input = layers.Input(name="positive", shape=(1280))
negative_input = layers.Input(name="negative", shape=(1280))

distances = DistanceLayer()(
    embedding(anchor_input),
    embedding(positive_input),
    embedding(negative_input),
)

model = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

# model.summary()

In [3]:
imgs = 2
pred = model.predict((np.random.rand(imgs,1280),np.random.rand(imgs,1280),np.random.rand(imgs,1280)))
# print(pred.shape)
print(pred)

(array([0.47569087, 0.4862771 ], dtype=float32), array([0.45308483, 0.47156262], dtype=float32))


In [4]:
features = {}
# load the features dictionary from the file
with open('features.pickle', 'rb') as handle:
    features = pickle.load(handle)

In [13]:
#convert the features dictionary to a tensorflow hash table
table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(list(features.keys()), list(features.values())), -1
    )

InvalidArgumentError: Expected shape [10000] for value, got [10000,1280] [Op:LookupTableImportV2]

In [8]:
#load strings from train_triplets.txt
train_triplets = np.loadtxt('./train_triplets.txt', dtype=str,delimiter = ' ')


dataset = tf.data.Dataset.from_tensor_slices(train_triplets)


@tf.function
def load_image(inputs):
    filenames = inputs
    print(filenames)

    print(features[filenames[0]]==None)
    print(features[filenames[0]])
    print(features[filenames[0]])
    print(features[filenames[0]])
    print(features[filenames[0]])
    print(features[filenames[0]])
    print(features[filenames[0]])
    print(features[filenames[0]])
    anchor = tf.cast(features[filenames[0]], tf.float32)
    positive = tf.cast(features[filenames[1]], tf.float32)
    negative = tf.cast(features[filenames[2]], tf.float32)

    output = (anchor,positive,negative)
    
    return output

dataset = dataset.map(load_image).cache().batch(32)

Tensor("inputs:0", shape=(3,), dtype=string)


TypeError: in user code:

    File "C:\Users\Timothe\AppData\Local\Temp/ipykernel_58168/1586639366.py", line 13, in load_image  *
        print(features[filenames[0]]==None)

    TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.


In [5]:
a,p,n = np.loadtxt('./train_triplets.txt', dtype=str,delimiter = ' ',unpack=True)
print(a.shape)

(59520,)


In [6]:


train_triplets = ([features[s] for s in a], [features[s] for s in p], [features[s] for s in n])
print(len(train_triplets))
print(len(train_triplets[0]))
print(len(train_triplets[0][0]))

3
59520
1280


In [7]:
dataset = tf.data.Dataset.from_tensor_slices(train_triplets)


In [8]:
dataset = dataset.cache().batch(batch_size).prefetch(300*batch_size)#.shuffle(300*batch_size)

In [9]:
next(iter(dataset))

(<tf.Tensor: shape=(24, 1280), dtype=float32, numpy=
 array([[ 2.01556921e-01,  8.89848650e-01,  2.10823193e-02, ...,
          2.08005682e-02,  2.65454412e-01,  6.79266036e-01],
        [-0.00000000e+00,  8.35452747e+00, -6.53548288e-13, ...,
         -3.37340017e-11, -0.00000000e+00, -1.12576436e-29],
        [-6.25388790e-03,  1.26523459e+00, -2.57090571e-10, ...,
          1.08705787e-03,  2.80684263e-01, -1.14109041e-03],
        ...,
        [-2.31602303e-06,  6.19625270e-01,  7.06529140e+00, ...,
          1.99381757e+00,  8.70376492e+00, -1.11546001e-06],
        [ 1.60655098e+01,  8.20239544e+00, -6.87726135e-12, ...,
         -0.00000000e+00, -0.00000000e+00,  5.75990772e+00],
        [-5.69873862e-03, -4.02146455e-10, -2.94570929e-30, ...,
          1.60260427e+00,  9.54223096e-01,  4.78756285e+00]], dtype=float32)>,
 <tf.Tensor: shape=(24, 1280), dtype=float32, numpy=
 array([[-1.8396137e-22,  3.2007507e+01,  2.0856602e+00, ...,
          2.5432057e+00, -7.8127780e-09, -1.2

In [10]:
# # This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
# def plotImages(images_arr):
#     fig, axes = plt.subplots(1, 5, figsize=(20,20))
#     axes = axes.flatten()
#     for img, ax in zip(images_arr, axes):
#         ax.imshow(img)
#     plt.tight_layout()
#     plt.show()

# augmented_images = [my_gen[0][0][0] for i in range(5)]
# plotImages(augmented_images)

In [11]:
class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]

In [12]:
siamese_model = SiameseModel(model)
siamese_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001))


In [13]:

#this is needed, otherwise nothing works
pred = siamese_model.predict(next(iter(dataset)))
print(pred)

(array([1.1430522, 1.5106184, 1.3340852, 1.6215323, 1.5394036, 1.2403878,
       1.3315356, 1.3004639, 1.307467 , 1.6082542, 1.1300997, 1.4777272,
       1.5612755, 0.9490143, 1.027336 , 1.4516199, 1.0350378, 1.3755169,
       1.4678793, 1.305803 , 1.6243707, 1.5846703, 1.5075547, 1.3554487],
      dtype=float32), array([1.1424536 , 1.443347  , 1.2928619 , 1.2390403 , 1.6826062 ,
       1.5334636 , 1.3321551 , 1.1854397 , 1.2038375 , 1.4775866 ,
       1.3961335 , 1.3130937 , 1.4303811 , 1.5570477 , 1.6615677 ,
       1.6806996 , 1.7886539 , 0.82114995, 1.6487105 , 1.3218422 ,
       1.6870714 , 1.2530386 , 1.4092016 , 1.355107  ], dtype=float32))


In [14]:
# print(my_gen.__getitem__(5)[0])

In [15]:
train_steps_per_epoch = int((59520)/batch_size)
# train_steps_per_epoch = int((59520*5)/batch_size)

# checkpoint_filepath = './checkpoint'
checkpoint_filepath = './checkpoints/checkpoint-{epoch}/'
# checkpoint_filepath = "saved-model-{epoch}.hdf5"
# checkpoint_filepath = "saved-model-{epoch}"
#save checkpoint after every epoch
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
)


siamese_model.fit(dataset, epochs=1,steps_per_epoch=train_steps_per_epoch,batch_size=batch_size,callbacks=[model_checkpoint_callback])
# siamese_model.fit(train_triplets, epochs=2,steps_per_epoch=train_steps_per_epoch,batch_size=batch_size,callbacks=[model_checkpoint_callback])


 674/2480 [=======>......................] - ETA: 8s - loss: 0.4719

In [ ]:

# model_path = "siamese_model"
# tf.saved_model.save(siamese_model, model_path)
